[View in Colaboratory](https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/master/DriveAPI.ipynb)

In [0]:

from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
#@title Directory id { run: "auto", vertical-output: true, output-height: 100, form-width: "50%", display-mode: "form" }
dir_id = "1kghdjPxiHRvQY2sQa1koli2csSGqni8_" #@param {type:"string"}
windows_fmt = False #@param {type:"boolean"}
# List .txt files in the root.
#

In [0]:

# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "'%s' in parents and mimeType = 'application/pdf'" % dir_id }).GetList()
doc_files = drive.ListFile({'q': "'%s' in parents and mimeType = 'application/vnd.google-apps.document'" % dir_id }).GetList()

debug = False
files=[]
row="{},{}.{}"
if windows_fmt:
  row+='\n'
for file in doc_files:
  files.append(file['id'])
  if debug:
    print(row.format(file['title'], file['id'], file['mimeType']))

print("Total Docs: {}".format(len(doc_files)))
doc_files[0:3]
doc_files[len(doc_files)-3:len(doc_files)]